In [1]:
import xgcm
import xarray as xr
import numpy as np
from xbasin import remap_vertical, compute_moc
from matplotlib import pyplot as plt
import cmocean as cm

In [2]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=1, memory_limit='1GB')
client

Client Scheduler: tcp://127.0.0.1:35189 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 1 Memory: 1000.00 MB


We open the example data and create 2 grids: 1 for the dataset we have and 1 for the remapped one.
Here '_fr' means *from* and '_to' *to* (i.e. remapped data).

In [3]:
ds = xr.open_dataset('data/nemo_output_ex.nc')

from xnemogcm import open_nemo_and_domain_cfg
ds = open_nemo_and_domain_cfg(datadir='/home/romain/Documents/Education/PhD/Courses/2019-OC6310/Project/Experiments/EXP_eos00/Rawdata')

In [4]:
ds.coords['exp'] = 'test'
ds = ds.expand_dims('exp')
ds = ds.chunk({'t':1})
ds

<xarray.Dataset>
Dimensions:        (axis_nbounds: 2, exp: 1, t: 5, x_c: 21, x_f: 21, y_c: 40, y_f: 40, z_c: 36, z_f: 36)
Coordinates:
  * z_f            (z_f) float64 -0.5 0.5 1.5 2.5 3.5 ... 31.5 32.5 33.5 34.5
  * t              (t) object 7756-01-01 00:00:00 ... 7796-01-01 00:00:00
  * x_c            (x_c) int64 0 1 2 3 4 5 6 7 8 ... 12 13 14 15 16 17 18 19 20
  * y_c            (y_c) int64 0 1 2 3 4 5 6 7 8 ... 31 32 33 34 35 36 37 38 39
  * z_c            (z_c) int64 0 1 2 3 4 5 6 7 8 ... 27 28 29 30 31 32 33 34 35
  * x_f            (x_f) float64 0.5 1.5 2.5 3.5 4.5 ... 17.5 18.5 19.5 20.5
  * y_f            (y_f) float64 0.5 1.5 2.5 3.5 4.5 ... 36.5 37.5 38.5 39.5
  * exp            (exp) <U4 'test'
Dimensions without coordinates: axis_nbounds
Data variables:
    depthw_bounds  (exp, z_f, axis_nbounds) float32 dask.array<chunksize=(1, 36, 2), meta=np.ndarray>
    t_bounds       (exp, t, axis_nbounds) object dask.array<chunksize=(1, 1, 2), meta=np.ndarray>
    woce           (exp, t, z_f, y_c, x_c) float32 dask.array<chunksize=(1, 1, 36, 40, 21), meta=np.ndarray>
    e3w            (exp, t, z_f, y_c, x_c) float32 dask.array<chunksize=(1, 1, 36, 40, 21), meta=np.ndarray>
    deptht_bounds  (exp, z_c, axis_nbounds) float32 dask.array<chunksize=(1, 36, 2), meta=np.ndarray>
    e3t            (exp, t, z_c, y_c, x_c) float32 dask.array<chunksize=(1, 1, 36, 40, 21), meta=np.ndarray>
    thetao         (exp, t, z_c, y_c, x_c) float32 dask.array<chunksize=(1, 1, 36, 40, 21), meta=np.ndarray>
    so             (exp, t, z_c, y_c, x_c) float32 dask.array<chunksize=(1, 1, 36, 40, 21), meta=np.ndarray>
    rhop           (exp, t, z_c, y_c, x_c) float32 dask.array<chunksize=(1, 1, 36, 40, 21), meta=np.ndarray>
    tos            (exp, t, y_c, x_c) float32 dask.array<chunksize=(1, 1, 40, 21), meta=np.ndarray>
    sos            (exp, t, y_c, x_c) float32 dask.array<chunksize=(1, 1, 40, 21), meta=np.ndarray>
    zos            (exp, t, y_c, x_c) float32 dask.array<chunksize=(1, 1, 40, 21), meta=np.ndarray>
    bn2            (exp, t, z_c, y_c, x_c) float32 dask.array<chunksize=(1, 1, 36, 40, 21), meta=np.ndarray>
    mldr10_1       (exp, t, y_c, x_c) float32 dask.array<chunksize=(1, 1, 40, 21), meta=np.ndarray>
    relvor         (exp, t, z_c, y_c, x_c) float32 dask.array<chunksize=(1, 1, 36, 40, 21), meta=np.ndarray>
    qsr            (exp, t, y_c, x_c) float32 dask.array<chunksize=(1, 1, 40, 21), meta=np.ndarray>
    qns            (exp, t, y_c, x_c) float32 dask.array<chunksize=(1, 1, 40, 21), meta=np.ndarray>
    empmr          (exp, t, y_c, x_c) float32 dask.array<chunksize=(1, 1, 40, 21), meta=np.ndarray>
    qt             (exp, t, y_c, x_c) float32 dask.array<chunksize=(1, 1, 40, 21), meta=np.ndarray>
    saltflx        (exp, t, y_c, x_c) float32 dask.array<chunksize=(1, 1, 40, 21), meta=np.ndarray>
    botpres        (exp, t, y_c, x_c) float32 dask.array<chunksize=(1, 1, 40, 21), meta=np.ndarray>
    depthu_bounds  (exp, z_c, axis_nbounds) float32 dask.array<chunksize=(1, 36, 2), meta=np.ndarray>
    e3u            (exp, t, z_c, y_c, x_f) float32 dask.array<chunksize=(1, 1, 36, 40, 21), meta=np.ndarray>
    uos            (exp, t, y_c, x_f) float32 dask.array<chunksize=(1, 1, 40, 21), meta=np.ndarray>
    uo             (exp, t, z_c, y_c, x_f) float32 dask.array<chunksize=(1, 1, 36, 40, 21), meta=np.ndarray>
    depthv_bounds  (exp, z_c, axis_nbounds) float32 dask.array<chunksize=(1, 36, 2), meta=np.ndarray>
    e3v            (exp, t, z_c, y_f, x_c) float32 dask.array<chunksize=(1, 1, 36, 40, 21), meta=np.ndarray>
    vos            (exp, t, y_f, x_c) float32 dask.array<chunksize=(1, 1, 40, 21), meta=np.ndarray>
    vo             (exp, t, z_c, y_f, x_c) float32 dask.array<chunksize=(1, 1, 36, 40, 21), meta=np.ndarray>
    nav_lon        (exp, y_c, x_c) float32 dask.array<chunksize=(1, 40, 21), meta=np.ndarray>
    nav_lat        (exp, y_c, x_c) float32 dask.array<chunksize=(1, 40, 21), meta=np.ndarray>
    jpiglo         (

In [5]:
metrics_fr = {
    ('X',): ['e1t', 'e1u', 'e1v', 'e1f'],
    ('Y',): ['e2t', 'e2u', 'e2v', 'e2f'],
    ('Z',): ['e3t', 'e3u', 'e3v', 'e3w']
}
metrics_to = {
    ('X',): ['e1t', 'e1u', 'e1v', 'e1f'],
    ('Y',): ['e2t', 'e2u', 'e2v', 'e2f'],
    ('Z',): ['e3t_1d', 'e3w_1d']
}
grid_fr = xgcm.Grid(ds, periodic=False, metrics=metrics_fr)
grid_to = xgcm.Grid(ds, periodic=False, metrics=metrics_to)

In [6]:
vo_to = remap_vertical(
    ds.vo,
    grid_fr,
    grid_to,
    axis='Z'
)

In [18]:
%time vo_to.compute()

CPU times: user 15 ms, sys: 0 ns, total: 15 ms
Wall time: 29.2 ms


<xarray.DataArray 'vo' (exp: 1, t: 5, z_c: 36, y_f: 40, x_c: 21)>
array([[[[[ 0.00000000e+00,  2.91885197e-03, -3.12995574e-02, ...,
           -9.92661739e-02, -5.63571376e-02,  0.00000000e+00],
          [ 0.00000000e+00, -2.91885197e-03,  3.12995574e-02, ...,
            9.92661739e-02,  5.63571376e-02,  0.00000000e+00],
          [ 0.00000000e+00, -1.25950670e-02,  4.18540421e-02, ...,
            1.33280351e-01,  9.36765071e-02,  0.00000000e+00],
          ...,
          [ 0.00000000e+00, -1.05439206e-03, -2.89586647e-03, ...,
           -7.28630450e-03,  5.54308801e-03,  0.00000000e+00],
          [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
            0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
          [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
            0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

         [[ 0.00000000e+00,  2.94672455e-03, -3.12574235e-02, ...,
           -5.14477625e-02, -1.76076593e-02,  0.00000000e+00],
          [ 0.00000000e+00, -2.94672455e-03,  3.12574235e-02, ...,
            5.14477625e-02,  1.76076593e-02,  0.00000000e+00],
          [ 0.00000000e+00, -1.26426384e-02,  4.16521033e-02, ...,
            1.21090617e-01,  4.72709177e-02,  0.00000000e+00],
          ...,
          [ 0.00000000e+00, -9.29476107e-04, -2.26030782e-03, ...,
           -6.94625359e-03,  5.64542321e-03,  0.00000000e+00],
          [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
            0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
          [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
            0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

         [[ 0.00000000e+00,  3.14595804e-03, -3.10253673e-02, ...,
            3.34319669e-02,  1.10752139e-02,  0.00000000e+00],
          [ 0.00000000e+00, -3.14595804e-03,  3.10253673e-02, ...,
           -3.34319669e-02, -1.10752139e-02,  0.00000000e+00],
          [ 0.00000000e+00, -1.27386145e-02,  4.13109914e-02, ...,
           -1.44797752e-03, -1.89006112e-02,  0.00000000e+00],
          ...,
          [ 0.00000000e+00, -5.31613159e-04, -1.32695513e-03, ...,
           -6.36782161e-03,  5.84448712e-03,  0.00000000e+00],
          [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
            0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
          [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
            0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

         ...,

         [[ 0.00000000e+00,  5.55861659e-04,  1.98149670e-04, ...,
           -6.88957386e-07, -2.15086508e-05,  0.00000000e+00],
          [ 0.00000000e+00, -5.55861659e-04, -1.98149670e-04, ...,
            6.88957386e-07,  2.15086508e-05,  0.00000000e+00],
          [ 0.00000000e+00, -2.00801020e-03, -6.98525287e-04, ...,
           -6.98680491e-06, -1.97244335e-05,  0.00000000e+00],
          ...,
          [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
            0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
          [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
            0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
          [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
            0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

         [[ 0.00000000e+00,  5.28441521e-04,  2.06383835e-04, ...,
            2.44190478e-05,  4.98352149e-05,  0.00000000e+00],
          [ 0.00000000e+00, -5.28441521e-04, -2.06383835e-04, ...,
           -2.44190478e-05, -4.98352149e-05,  0.00000000e+00],
          [ 0.00000000e+00, -1.05904619e-03, -5.57242852e-04, ...,
           -1.44400912e-05, -7.42988861e-05,  0.00000000e+00],
          ...,
          [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
            0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
          [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
            0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
          [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,

In [ ]:
from xbasin.eos import compute_rho

rho = compute_rho(
    grid_fr.interp(ds.thetao,axis='Y', boundary='extrapolate'),
    grid_fr.interp(ds.so, axis='Y', boundary='extrapolate'),
    0,
    lambda1=4.9520e-2
) - 1000

ds['sigmav'] = rho

In [ ]:
from xbasin import create_density_grid

ds = create_density_grid(
    ds,
    dd=0.0001,
    rho=ds['sigmav']
)

ds = ds.chunk({'t':1,'s_c':100, 's_f':100})

metrics = {
    ('X',): ['e1t', 'e1u', 'e1v', 'e1f'],
    ('Y',): ['e2t', 'e2u', 'e2v', 'e2f'],
    ('Z',): ['e3t', 'e3u', 'e3v', 'e3w'],
    ('S',): ['e3t_s_1d', 'e3w_s_1d']
}
grid = xgcm.Grid(ds, periodic=False, metrics=metrics)
print(grid)
ds

In [ ]:
from xbasin._stream_functions import _compute_moc_in_density_coordinates

psi = _compute_moc_in_density_coordinates(
    v=ds.vo,
    rho=ds.sigmav,
    density_levels=ds.grhow_0,
    grid=grid,
    X='X',
    Z='Z'
)

In [ ]:
psi.compute()